In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import random 
import numpy as np
import json
import pickle
import nltk 
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation, Dropout,LSTM,Embedding
from tensorflow.keras.optimizers import SGD

In [3]:
lemmatizer = WordNetLemmatizer()

In [4]:
intents = json.loads(open('/content/drive/MyDrive/datasets/intents.json').read())

In [52]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
words = []
classes = []
documents = []
ignore_letters = ['?','/','.',',']
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern) #basically converts string to list
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes :
      classes.append(intent['tag'])



In [ ]:
words

In [ ]:
documents

In [56]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [6]:
classes

['greeting',
 'bored',
 'about_me',
 'info_career',
 'relationship',
 'goodbye',
 'stocks']

In [8]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [9]:
training = []
output_empty = [0] * len(classes)
for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns ]
  for word in words :
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1 
  training.append([bag,output_row])

In [10]:
random.shuffle(training)
training = np.array(training)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [11]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [12]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0],),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation = 'softmax'))

In [13]:
sgd = SGD(lr=0.01,decay = 1e-6 , momentum = 0.9 , nesterov=True)
model.compile(loss = "categorical_crossentropy",optimizer = sgd , metrics=['accuracy'])

In [14]:
model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose = 5)

Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
Epoch 70/200
Epoch 71/200
Epoch 72/200
Epoch 73/200
Epoch 74/200
Epoch 75/200
Epoch 76/200
Epoch 77/200
Epoch 78

In [15]:
model.save('chatbot.h5')

In [16]:
from tensorflow.keras.models import load_model
model = load_model('chatbot.h5')

In [17]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words 

In [18]:
def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w :
        bag[i] = 1
  return np.array(bag)

In [19]:
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD ]

  results.sort(key=lambda x : x[1],reverse = True)
  return_list = []
  for r in results : 
    return_list.append({'intent':classes[r[0]] , 'probablity' : str(r[1])})
  return return_list

In [20]:
def get_response(intents_list,intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents : 
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

In [21]:
classes

['greeting',
 'bored',
 'about_me',
 'info_career',
 'relationship',
 'goodbye',
 'stocks']

In [24]:
while True:
  message = input("")
  ints = predict_class(message)
  res = get_response(ints, intents)
  print(res)

i am sad
Talk to you later


KeyboardInterrupt: ignored